In [1]:
!pip install python-telegram-bot --upgrade

  Attempting uninstall: tornado
    Found existing installation: tornado 6.0.4
    Uninstalling tornado-6.0.4:
      Successfully uninstalled tornado-6.0.4


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\Karolina\\anaconda3\\Lib\\site-packages\\~ornado\\speedups.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



# Дни 1 и 2

In [12]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [13]:
BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': ['привет!', 'хай!!', 'прив'],
            'responses': ['здравствуйте', 'хэллоу!', 'хей!!1']
        },
        'bye': {
            'examples': ['пока!', 'покеда!!', 'до свиданья'],
            'responses': ['увидимся))', 'до связи', 'Сайонара']
        }
    }
}

In [14]:
def clean(text):
  clean_text = ''
  for ch in text.lower():
    if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
      clean_text = clean_text + ch
  return clean_text

In [16]:
# with open('BOT_CONFIG.json', 'r') as f:
#   BOT_CONFIG = json.load(f)
print(BOT_CONFIG)
#del BOT_CONFIG['intents']['price']
#len(BOT_CONFIG['intents'].keys())

{'intents': {'hello': {'examples': ['привет!', 'хай!!', 'прив'], 'responses': ['здравствуйте', 'хэллоу!', 'хей!!1']}, 'bye': {'examples': ['пока!', 'покеда!!', 'до свиданья'], 'responses': ['увидимся))', 'до связи', 'Сайонара']}}}


In [17]:
texts = []
y = []
for intent in BOT_CONFIG['intents'].keys():
  for example in BOT_CONFIG['intents'][intent]['examples']:
    texts.append(example)
    y.append(intent)
len(texts), len(y)

(6, 6)

In [18]:
train_texts, test_texts, y_train, y_test = train_test_split(texts, y, random_state=42, test_size=0.2)

In [19]:
# vectorizer = CountVectorizer(ngram_range=(1,3), analyzer='char_wb') #TfidfVectorizer(ngram_range=(1,5), analyzer='char_wb')
vectorizer = CountVectorizer(ngram_range=(2, 2), min_df=0.1, max_df=0.7, 
                             max_features=100, analyzer='char_wb') 
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [20]:
vocab = vectorizer.get_feature_names_out()
len(vocab)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names_out'

In [ ]:
# clf = RandomForestClassifier(n_estimators=300) #LogisticRegression().fit(X_train, y_train)
# clf = AdaBoostClassifier(n_estimators=5000, random_state=0)
clf = KNeighborsClassifier(n_neighbors=1, weights = 'distance', 
                           algorithm='brute', leaf_size=30, p=1, 
                           metric='minkowski', metric_params=None, n_jobs=-1)
clf.fit(X_train, y_train)
clf.score(X_train, y_train), clf.score(X_test, y_test) #LogisticRegression: 0.14893617021276595, RandomForestClassifier: 0.19574468085106383

(0.6556503198294243, 0.0425531914893617)

In [ ]:
# clf.predict(vectorizer.transform(['привет!']))

In [ ]:
# def get_intent(text):
#   for intent in BOT_CONFIG['intents'].keys():
#     for example in BOT_CONFIG['intents'][intent]['examples']:
#       s1 = clean(text)
#       s2 = clean(example)
#       if nltk.edit_distance(s1, s2) / max(len(s1), len(s2)) < 0.4:
#         return intent
#   return 'интент не найден'

def get_intent_by_model(text):
  return clf.predict(vectorizer.transform([text]))[0]

def bot(input_text):
  intent = get_intent_by_model(input_text)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
input_text = ''
while input_text != 'stop':
  input_text = input()
  if input_text != 'stop':
    response = bot(input_text)
    print(response)

stop


# День 3

In [ ]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    input_text = update.message.text
    output_text = bot(input_text)
    update.message.reply_text(output_text)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("5010629338:AAGRKszcyoCUF067nljwOiR7YVGaDmBttJo")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2022-01-12 18:48:08,296 - apscheduler.scheduler - INFO - Scheduler started
2022-01-12 19:00:57,502 - telegram.ext.updater - ERROR - Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2022-01-12 19:00:57,505 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/updater.py", line 646, in _network_loop_retry
    if not action_cb():
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/updater.py", line 601, in polling_action_cb
    allowed_updates=allowed_updates,
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/extbot.py", line 230, in get_updates
    api_kwargs=api_kwargs,
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 130, in decorator
    result = func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 3058,